In [ ]:
pip install git+https://github.com/facebookresearch/demucs

1. Install required packages

%pip install librosa
%pip install noisereduce
%pip install soundfile

In [ ]:
from azure.storage.blob import BlobServiceClient
import os
import subprocess
import shutil

# === CONFIGURATION ===
AZURE_STORAGE_CONNECTION_STRING = os.getenv("TBD")
input_container_name = os.getenv("TBD")
output_container_name = os.getenv("TBD")

# === SETUP BLOB CLIENTS ===
blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
input_container = blob_service_client.get_container_client(input_container_name)
output_container = blob_service_client.get_container_client(output_container_name)

# === LOCAL FOLDERS ===
TEMP_AUDIO_DIR = "temp_audio"
DEMUC_OUTPUT_DIR = "demucs_out"
os.makedirs(TEMP_AUDIO_DIR, exist_ok=True)

# === PROCESS ALL .mp3 FILES ===
for blob in input_container.list_blobs():
    if blob.name.endswith(".mp3"):
        print(f"\n▶️ Processing: {blob.name}")
        audio_blob_name = blob.name
        local_audio_path = os.path.join(TEMP_AUDIO_DIR, os.path.basename(audio_blob_name))

        # Download MP3 file
        with open(local_audio_path, "wb") as download_file:
            download_stream = input_container.download_blob(audio_blob_name)
            download_file.write(download_stream.readall())

        # Run Demucs on the MP3 file
        subprocess.run([
            "demucs",
            "--two-stems", "vocals",  # Only extract vocals
            "--out", DEMUC_OUTPUT_DIR,
            local_audio_path
        ], check=True)

        # Extract path to Demucs-separated vocals
        base_name = os.path.splitext(os.path.basename(audio_blob_name))[0]
        vocals_path = os.path.join(DEMUC_OUTPUT_DIR, "htdemucs", base_name, "vocals.wav")

        # Upload vocals to output container
        output_blob_name = audio_blob_name.replace(".mp3", "_clean.wav")
        with open(vocals_path, "rb") as data:
            output_container.upload_blob(name=output_blob_name, data=data, overwrite=True)
            print(f"✅ Uploaded: {output_blob_name}")

        # Cleanup temp files
        os.remove(local_audio_path)
        shutil.rmtree(os.path.join(DEMUC_OUTPUT_DIR, "htdemucs", base_name), ignore_errors=True)

print("\n✅ All audio files cleaned and uploaded.")
